# APIM ❤️ AI Agents

## Model Context Protocol (MCP) from REST API lab
![flow](../../images/model-context-protocol.gif)

Playground to transform an existing REST API to the [Model Context Protocol](https://modelcontextprotocol.io/) with Azure API Management. 

### Prerequisites

- [Python 3.12 or later version](https://www.python.org/) installed
- [VS Code](https://code.visualstudio.com/) installed with the [Jupyter notebook extension](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter) enabled
- [Python environment](https://code.visualstudio.com/docs/python/environments#_creating-environments) with the [requirements.txt](../../requirements.txt) or run `pip install -r requirements.txt` in your terminal
- [An Azure Subscription](https://azure.microsoft.com/free/) with [Contributor](https://learn.microsoft.com/en-us/azure/role-based-access-control/built-in-roles/privileged#contributor) + [RBAC Administrator](https://learn.microsoft.com/en-us/azure/role-based-access-control/built-in-roles/privileged#role-based-access-control-administrator) or [Owner](https://learn.microsoft.com/en-us/azure/role-based-access-control/built-in-roles/privileged#owner) roles
- [Azure CLI](https://learn.microsoft.com/cli/azure/install-azure-cli) installed and [Signed into your Azure subscription](https://learn.microsoft.com/cli/azure/authenticate-azure-cli-interactively)

▶️ Click `Run All` to execute all steps sequentially, or execute them `Step by Step`...


<a id='0'></a>
### 0️⃣ Initialize notebook variables

- Resources will be suffixed by a unique string based on your subscription id.
- Adjust the location parameters according your preferences and on the [product availability by Azure region.](https://azure.microsoft.com/explore/global-infrastructure/products-by-region/?cdn=disable&products=cognitive-services,api-management) 


In [ ]:
import os, sys, json
sys.path.insert(1, '../../shared')  # add the shared directory to the Python path
import utils

subscription_id = utils.get_current_subscription()
deployment_name = os.path.basename(os.path.dirname(globals()['__vsc_ipynb_file__']))
resource_group_name = f"lab-{deployment_name}" # change the name to match your naming style or an existing resource group
resource_group_location = "ukwest"

aiservices_config = [{"name": "foundry1", "location": "uksouth"}]

models_config = [{"name": "gpt-4.1-mini", "publisher": "OpenAI", "version": "2025-04-14", "sku": "GlobalStandard", "capacity": 20}]

apim_sku = 'Basicv2'
apim_subscriptions_config = [{"name": "subscription1", "displayName": "Subscription 1"}]

inference_api_path = "inference"  # path to the inference API in the APIM service
inference_api_type = "AzureOpenAI"  # options: AzureOpenAI, AzureAI, OpenAI, PassThrough
inference_api_version = "2025-03-01-preview"
foundry_project_name = deployment_name

apic_location = "uksouth"  # location for the API Center service
apic_service_name_prefix = 'apic6'

utils.print_ok('Notebook initialized')


<a id='1'></a>
### 1️⃣ Create deployment using 🦾 Bicep

This lab uses [Bicep](https://learn.microsoft.com/azure/azure-resource-manager/bicep/overview?tabs=bicep) to declarative define all the resources that will be deployed in the specified resource group. Change the parameters or the [main.bicep](main.bicep) directly to try different configurations. 

In [ ]:
# Create the resource group if doesn't exist
utils.create_resource_group(resource_group_name, resource_group_location)

# Define the Bicep parameters
bicep_parameters = {
    "$schema": "https://schema.management.azure.com/schemas/2019-04-01/deploymentParameters.json#",
    "contentVersion": "1.0.0.0",
    "parameters": {
        "apimSku": { "value": apim_sku },
        "aiServicesConfig": { "value": aiservices_config },
        "modelsConfig": { "value": models_config },
        "apimSubscriptionsConfig": { "value": apim_subscriptions_config },
        "inferenceAPIPath": { "value": inference_api_path },
        "inferenceAPIType": { "value": inference_api_type },
        "foundryProjectName": { "value": foundry_project_name },
        "apicLocation": { "value": apic_location },
        "apicServiceNamePrefix": { "value": apic_service_name_prefix },
    }
}

# Write the parameters to the params.json file
with open('params.json', 'w') as bicep_parameters_file:
    bicep_parameters_file.write(json.dumps(bicep_parameters))

# Run the deployment
output = utils.run(f"az deployment group create --name {deployment_name} --resource-group {resource_group_name} --template-file main.bicep --parameters params.json",
    f"Deployment '{deployment_name}' succeeded", f"Deployment '{deployment_name}' failed")

<a id='2'></a>
### 2️⃣ Get the deployment outputs

Retrieve the required outputs from the Bicep deployment.

In [ ]:
# Obtain all of the outputs from the deployment
output = utils.run(f"az deployment group show --name {deployment_name} -g {resource_group_name}", f"Retrieved deployment: {deployment_name}", f"Failed to retrieve deployment: {deployment_name}")

if output.success and output.json_data:
    log_analytics_id = utils.get_deployment_output(output, 'logAnalyticsWorkspaceId', 'Log Analytics Id')
    apim_service_id = utils.get_deployment_output(output, 'apimServiceId', 'APIM Service Id')
    apim_resource_gateway_url = utils.get_deployment_output(output, 'apimResourceGatewayURL', 'APIM API Gateway URL')
    apim_subscriptions = json.loads(utils.get_deployment_output(output, 'apimSubscriptions').replace("\'", "\""))
    for subscription in apim_subscriptions:
        subscription_name = subscription['name']
        subscription_key = subscription['key']
        utils.print_info(f"Subscription Name: {subscription_name}")
        utils.print_info(f"Subscription Key: ****{subscription_key[-4:]}")
    api_key = apim_subscriptions[0].get("key") # default api key to the first subscription key
    foundry_project_endpoint = utils.get_deployment_output(output, 'foundryProjectEndpoint', 'Foundry Project Endpoint')


<a id='testconnection'></a>
### 🧪 Test the connection to the MCP servers and List Tools



In [ ]:
import nest_asyncio
import asyncio
nest_asyncio.apply()

from mcp import ClientSession
from mcp.client.streamable_http import streamablehttp_client


async def list_tools(server_url):
    # Connect to a streamable HTTP server
    async with streamablehttp_client(server_url) as (
        read_stream,
        write_stream,
        _,
    ):
        # Create a session using the client streams
        async with ClientSession(read_stream, write_stream) as session:
            # Initialize the connection
            await session.initialize()
            # List available tools
            tools = await session.list_tools()
            print(f"Available tools for {server_url}: {[tool.name for tool in tools.tools]}")
            

if __name__ == "__main__":
    asyncio.run(list_tools(f"{apim_resource_gateway_url}/weather-mcp/mcp"))
    asyncio.run(list_tools(f"{apim_resource_gateway_url}/ms-learn-mcp"))



<a id='inspector'></a>
### 🧪 (optional) Use the [MCP Inspector](https://modelcontextprotocol.io/docs/tools/inspector) for testing and debugging the MCP servers

#### Execute the following steps:
1. Execute `npx @modelcontextprotocol/inspector` in a terminal
2. Open the provided URL in a browser
3. Set the transport type as streamable http
4. Provide the MCP server url and click connect
5. Select the "Tools" tab to see and run the available tools

<a id='openaiagent'></a>
### 🧪 Run an OpenAI Agent with MCP tools



In [ ]:
import asyncio

from openai import AsyncAzureOpenAI
from agents import Agent, Runner, set_default_openai_client
from agents.mcp import MCPServerStreamableHttp
from agents.model_settings import ModelSettings


async def run_agent(server_url: str):
    client = AsyncAzureOpenAI(azure_endpoint=f"{apim_resource_gateway_url}/{inference_api_path}",
                                api_key=api_key,
                                api_version=inference_api_version)
    set_default_openai_client(client)

    async with MCPServerStreamableHttp(
        name="Streamable HTTP Weather Server",
        params={
            "url": server_url,
            "headers": {
                "agent-id": "OpenAIAgent"
            }            
        },
    ) as server:
        extra_headers = {"agent-id": "OpenAIAgent"}
        agent = Agent(
            name="Assistant",
            instructions="Use the tools to answer the questions.",
            mcp_servers=[server],
            model_settings=ModelSettings(tool_choice="required", extra_headers=extra_headers),
            model=models_config[0]['name']            
        )

        # Run the `get_weather` tool
        message = "What's the weather in Lisbon?"
        print(f"Running: {message}")
        result = await Runner.run(starting_agent=agent, input=message)
        print(result.final_output)

if __name__ == "__main__":
    asyncio.run(run_agent(f"{apim_resource_gateway_url}/weather-mcp/mcp"))



<a id='Azure AI Agents'></a>
### 🧪 Execute an [Azure AI Foundry Agent using MCP Tools](https://learn.microsoft.com/en-us/azure/ai-foundry/agents/how-to/tools/model-context-protocol) via Azure API Management

In [ ]:
# Copyright (c) Microsoft. All rights reserved.

import asyncio

from azure.ai.agents.models import McpTool
from azure.identity.aio import DefaultAzureCredential

from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings, AzureAIAgentThread
from semantic_kernel.contents import ChatMessageContent, FunctionCallContent, FunctionResultContent

TASK = "What's the current weather in Lisbon?"

async def handle_intermediate_messages(message: ChatMessageContent) -> None:
    for item in message.items or []:
        if isinstance(item, FunctionResultContent):
            print(f"Function Result:> {item.result} for function: {item.name}")
        elif isinstance(item, FunctionCallContent):
            print(f"Function Call:> {item.name} with arguments: {item.arguments}")
        else:
            print(f"{item}")


async def main() -> None:
    async with (
        DefaultAzureCredential() as creds,
        AzureAIAgent.create_client(credential=creds) as client,
    ):
        # 1. Define the MCP tool with the server URL
        mcp_tool = McpTool(
            server_label="weather",
            server_url=f"{apim_resource_gateway_url}/weather-mcp/mcp",
            allowed_tools=[],  # Specify allowed tools if needed
        )

        # Optionally you may configure to require approval
        # Allowed values are "never" or "always"
        mcp_tool.set_approval_mode("never")

        agent_settings = AzureAIAgentSettings(
            model_deployment_name=models_config[0]['name']
        )

        # 2. Create an agent with the MCP tool on the Azure AI agent service
        agent_definition = await client.agents.create_agent(
            model=agent_settings.model_deployment_name,
            tools=mcp_tool.definitions,
            instructions="You are a helpful agent that can use MCP tools to assist users.",
        )

        # 3. Create a Semantic Kernel agent for the Azure AI agent
        agent = AzureAIAgent(
            client=client,
            definition=agent_definition,
        )

        # 4. Create a thread for the agent
        # If no thread is provided, a new thread will be
        # created and returned with the initial response
        thread: AzureAIAgentThread | None = None

        print(f"# User: '{TASK}'")
        # 5. Invoke the agent for the specified thread for response
        async for response in agent.invoke(
            messages=TASK, thread=thread, on_intermediate_message=handle_intermediate_messages
        ):
            print(f"# Agent: {response}")



if __name__ == "__main__":
    asyncio.run(main())

<a id='testratelimit'></a>
### 🧪 Test the rate limit on Microsoft Learn MCP pass-through

Check the [policy](src/ms-learn/mcp-server/policy.xml)

In [ ]:
# Unauthenticated call should fail with 401 Unauthorized
import requests

# apim_debug_authorization = utils.get_debug_credentials(apim_service_id, 'ms-learn-mcp')

request = {"method":"tools/call","params":{"name":"microsoft_docs_search","arguments":{"query":"Microsoft/Azure API Management","question":"how to configure the rate limit"}},"jsonrpc":"2.0","id":1}

for i in range(1):  
    response = requests.post(f"{apim_resource_gateway_url}/ms-learn-mcp", stream=True, headers={"Content-Type": "application/json", "agent-id": "Agent1", 
                                                                                                 # 'Apim-Debug-Authorization': apim_debug_authorization
                                                                                                 }, json=request)
    if response.status_code == 200:
        utils.print_ok(f"Run {i+1}. Received status code 200 as expected")
    else:
        utils.print_error(f"Run {i+1}. Unexpected status code: {response.status_code}. Response text: {response.text}")

    # print(json.dumps(utils.get_trace(apim_service_id, response.headers.get("Apim-Trace-Id")), indent=4)) 

    response.close()


<a id='unauthorizedtest'></a>
### 🧪 Test the Product Catalog MCP Authorization **WITHOUT** a valid token

Check the [policy](src/product-catalog/mcp-server/policy.xml)

In [ ]:
# Unauthenticated call should fail with 401 Unauthorized
import requests
utils.print_info("Calling sse endpoint WITHOUT authorization...")
request = {"method":"tools/call","params":{"name":"get-product-details","arguments":{"category":"electronics"}},"jsonrpc":"2.0","id":1}
response = requests.post(f"{apim_resource_gateway_url}/catalog-mcp/mcp", stream=True, headers={"Content-Type": "application/json"}, json=request)
if response.status_code == 401:
    utils.print_ok("Received 401 Unauthorized as expected")
elif response.status_code == 200:
    utils.print_error("Call succeeded. Double check that validate-jwt policy has been deployed to sse endpoint")
else:
    utils.print_error(f"Unexpected status code: {response.status_code}")
response.close()


<a id='authorizedtest'></a>
### 🧪 Test the Product Catalog MCP authorization **WITH** a valid token

Check the [policy](src/product-catalog/mcp-server/policy.xml)

In [ ]:
# authorized call should succeed and display the response
import requests, json
request = {"method":"tools/call","params":{"name":"get-product-details","arguments":{"category":"electronics"}},"jsonrpc":"2.0","id":1}

utils.print_info("Calling MCP endpoint WITH authorization...")
output = utils.run("az account get-access-token --resource \"https://azure-api.net/authorization-manager\"")
if output.success and output.json_data:
    access_token = output.json_data['accessToken']
    response = requests.post(f"{apim_resource_gateway_url}/catalog-mcp/mcp", stream=True, headers={"Content-Type": "application/json", "agent-id": "Agent1", "Authorization": "Bearer " + str(access_token)}, json=request)
    if response.status_code == 200:
        utils.print_ok("Received status code 200 as expected")

        print("Response:")
        for line in response.iter_lines(decode_unicode=True):
            if line:
                if (line == 'event: close'):
                    response.close()
                    break
                if (line.startswith('data')):
                    data = json.loads(line.strip()[5:])
                    print(json.dumps(json.loads(data["result"]["content"][0]["text"]), indent=4))
    else:
        utils.print_error(f"Unexpected status code: {response.status_code}. Response text: {response.text}")




<a id='authorizedtest'></a>
### 🧪 Test the Place Order MCP authorization **WITH** a valid token

Check the [policy](src/place-order/mcp-server/policy.xml)

In [ ]:
import requests

request = {"method":"tools/call","params":{"name":"PlaceOrder-invoke","arguments":{"request-PlaceOrder":{"sku":"sku-123","quantity":5}}},"jsonrpc":"2.0","id":1}

utils.print_info("Calling MCP endpoint WITH authorization...")
output = utils.run("az account get-access-token --resource \"https://azure-api.net/authorization-manager\"")
if output.success and output.json_data:
    access_token = output.json_data['accessToken']
    response = requests.post(f"{apim_resource_gateway_url}/order-mcp/mcp", stream=True, headers={"Content-Type": "application/json", "agent-id": "Agent1", "Authorization": "Bearer " + str(access_token)}, json=request)
    if response.status_code == 200:
        utils.print_ok("Received status code 200 as expected")

        print("Response:")
        for line in response.iter_lines(decode_unicode=True):
            if line:
                if (line == 'event: close'):
                    response.close()
                    break
                if (line.startswith('data')):
                    data = json.loads(line.strip()[5:])
                    print(json.dumps(json.loads(data["result"]["content"][0]["text"]), indent=4))
    else:
        utils.print_error(f"Unexpected status code: {response.status_code}. Response text: {response.text}")


<a id='clean'></a>
### 🗑️ Clean up resources

When you're finished with the lab, you should remove all your deployed resources from Azure to avoid extra charges and keep your Azure subscription uncluttered.
Use the [clean-up-resources notebook](clean-up-resources.ipynb) for that.